In [6]:
%matplotlib inline
import obspy
from obspy import read
import glob
import os, sys
import librosa
import numpy as np
from scipy import stats
import scipy as sp
import scipy.signal as signal
import pandas as pd
import matplotlib.pyplot as plt
import librosa.display
from sklearn.preprocessing import StandardScaler

random_state = 6
np.random.seed(random_state)

In [36]:
def extract_features(seismogram, signal_label):
    seis = seismogram.stats['sac']
    seis_id = str(str(seis['knetwk']) +'.'+ str(seis['kstnm']) +'.'+ str(seis['kcmpnm']) +'.'+ 'M.'+ str(seis['nzyear']) +'.'+  str(seis['nzjday']) +'.'+ str(seis['nzhour']) + str(seis['nzmin']) + str(seis['nzsec']))
    
    data = signal.detrend(seismogram.data)
    sample_rate = seismogram.stats.sampling_rate
    stft = np.abs(librosa.stft(data))
    mfccs = librosa.feature.mfcc(y = data, sr=sample_rate, n_mfcc=40)
    mfccs_mean = np.mean(mfccs.T, axis = 0)
    # mfccs_delta = np.mean(librosa.feature.delta(mfccs).T, axis=0)
    
    # 12 chroma
    chroma = librosa.feature.chroma_stft(S = stft, sr = sample_rate)
    chroma_mean = np.mean(chroma.T, axis = 0)
    
    ## mel from melspectrogram
    mel = librosa.feature.melspectrogram(data, sr = sample_rate)
    mel_max = np.amax(mel)
    mel_mean = np.mean(mel)
    power_max = np.amax(librosa.power_to_db(mel, ref=np.max))
    power_mean = np.amax(librosa.power_to_db(mel, ref=np.max))
    
    ## spectral centroid
    S, phase = librosa.magphase(stft)
    centiroid_mean =np.mean(librosa.feature.spectral_centroid(S=S)) # 1 feature
    centiroid_max =np.amax(librosa.feature.spectral_centroid(S=S)) # 1 feature
    centiroid_min =np.amin(librosa.feature.spectral_centroid(S=S)) # 1 feature
    
    max_amplitude = np.amax(S)
    mean_amplitude = np.mean(S)
    dbamplitude= np.mean(librosa.amplitude_to_db(S, ref=np.max).T, axis = 0) # 1025 features
    
    # spectral density
    f, Pxx_den = signal.periodogram(data, sample_rate)
    max_psd = np.sqrt(Pxx_den.max())
    
    f, Pxx_spec = signal.welch(data, sample_rate)
    welch_max_psd = np.sqrt(Pxx_spec.max())
    
    #RMSE 
    rmse_max = np.amax(librosa.feature.rmse(S=S))
    rmse_mean = np.mean(librosa.feature.rmse(S=S))
    
    ## statistical parameters
    moment = sp.stats.moment(data)
    variation = sp.stats.variation(data)
    skew = sp.stats.skew(data)
    var = np.var(data)
    autocr = np.correlate(data, data)
    kurto = sp.stats.kurtosis(data)
    
    features = np.hstack([seis_id, mfccs_mean, chroma_mean, mel_max, mel_mean, power_max, power_mean, centiroid_mean, centiroid_max, centiroid_min, max_amplitude, mean_amplitude, max_psd, welch_max_psd, rmse_max, rmse_mean, moment, variation, skew, var, autocr, kurto, signal_label])
    
    return features

def parse_and_stack_seismograms(seismograms, label):
    
    features = np.empty((0,73))
    
    if label == 'earthquake':
        target = 0
    elif label == 'explosion':
        target = 1

    for i, seismogram in enumerate(seismograms):
        single_feature= extract_features(seismogram, target)
        features = np.vstack([features, single_feature])
        
    dataFrame = pd.DataFrame(features)
    return dataFrame

In [20]:
earthquakes = read('./seismogram_v2/earthquakes/*.SAC')
explosions = read('./seismogram_v2/explosions/*.SAC')

/usr/local/lib/python3.6/site-packages/obspy/core/trace.py:187: UserWarning: Calibration factor set to 0.0!
  warnings.warn(msg, UserWarning)


In [ ]:
df_explosions  = parse_and_stack_seismograms(explosions, 'explosion')
df_earthquakes  = parse_and_stack_seismograms(earthquakes, 'earthquake')

/usr/local/lib/python3.6/site-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
/usr/local/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 50, using nperseg = 50
  .format(nperseg, input_length))
/usr/local/lib/python3.6/site-packages/scipy/stats/stats.py:977: RuntimeWarning: divide by zero encountered in float_scalars
  return a.std(axis) / a.mean(axis)
/usr/local/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 174, using nperseg = 174
  .format(nperseg, input_length))
/usr/local/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 126, using nperseg = 126
  .format(nperseg, input_length))
/usr/local/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserW

In [3]:
parent_dir_explosions = 'seismogram_v2/explosions/'
sub_dirs_explosions = ['1998-05-11-mb52-india','1998-05-28-mb48-pakistan', '1998-05-30-mb46-pakistan', '2013-02-12-mb51-north-korea', '2016-01-06-mb51-north-korea', '2017-09-03-mb63-north-korea']
parent_dir_earthquake = 'seismogram_v2/earthquakes/'
sub_dirs_earthquake = ['2004-12-26-mw90-sumatra', '2010-03-12-mw55-myanmar-india-border-region', '2017-08-15-mb49-southeast-of-ryukyu-islands', '2017-09-08-mww81-near-coast-of-chiapas-mexico', '2017-10-24-mww67-banda-sea']

## Explosions
print('--- Earthquake-----: ')
total = 0.0
for sub_dir in sub_dirs_earthquake:
    counter = len(glob.glob(os.path.join(parent_dir_earthquake, sub_dir, '*.SAC')))
    total += counter
    print('{}: {}'.format(sub_dir, counter))
print('Total seismograms: {}'. format(total))


## Explosions
print('--- Explosions-----: ')
total = 0.0
for sub_dir in sub_dirs_explosions:
    counter = len(glob.glob(os.path.join(parent_dir_explosions, sub_dir, '*.SAC')))
    total += counter
    print('{}: {}'.format(sub_dir, counter))
print('Total seismograms: {}'. format(total))

--- Earthquake-----: 
2004-12-26-mw90-sumatra: 1193
2010-03-12-mw55-myanmar-india-border-region: 3021
2017-08-15-mb49-southeast-of-ryukyu-islands: 2644
2017-09-08-mww81-near-coast-of-chiapas-mexico: 2722
2017-10-24-mww67-banda-sea: 3382
Total seismograms: 12962.0
--- Explosions-----: 
1998-05-11-mb52-india: 459
1998-05-28-mb48-pakistan: 470
1998-05-30-mb46-pakistan: 398
2013-02-12-mb51-north-korea: 3763
2016-01-06-mb51-north-korea: 2640
2017-09-03-mb63-north-korea: 2804
Total seismograms: 10534.0


In [12]:
def get_column_names():
    features = {
        'id':1,
        'mfccs': 40,
        'chroma': 12,
        'mel_max':1,
        'mel_mean':1,
        'power_max':1,
        'power_mean':1,
        'centiroid_mean': 1,
        'centiroid_max':1,
        'centiroid_min':1,
        'max_amplitude':1, 
        'mean_amplitude':1,
        'max_psd':1,
        'welch_max_psd':1,
        'rmse_max':1,
        'rmse_mean':1,
        'moment': 1,
        'variation': 1, 
        'skew': 1, 
        'var': 1, 
        'autocr': 1, 
        'kurto': 1, 
        'target': 1
    }
    
    names = list(features.keys())
    val = list(features.values())
    
    columns = []
    
    for i in range(len(features)):
        if val[i] > 1:
            for j in range(val[i]):
                columns.append(str(names[i])+'_'+ str(j))
                
        else:
            columns.append(str(names[i]))
    
    return columns

In [13]:
frames = [df_explosions, df_earthquakes]
df = pd.concat(frames)
df.columns = get_column_names()
df.describe()

,mfccs_0,mfccs_1,mfccs_2,mfccs_3,mfccs_4,mfccs_5,mfccs_6,mfccs_7,mfccs_8,mfccs_9,...,welch_max_psd,rmse_max,rmse_mean,moment,variation,skew,var,autocr,kurto,target
count,23460.000000,23460.000000,23460.000000,23460.000000,23460.000000,23460.000000,23460.000000,23460.000000,23460.000000,23460.000000,...,2.346000e+04,2.346000e+04,2.346000e+04,23460.0,2.345800e+04,23460.000000,2.346000e+04,2.346000e+04,23460.000000,23460.000000
mean,831.015422,74.997236,-2.795191,53.061141,0.768855,34.751700,6.163632,22.446028,10.836329,15.055261,...,5.550137e+04,5.724160e+06,1.488110e+06,0.0,inf,-0.031473,5.214679e+11,3.719397e+16,7.331627,0.447485
std,193.899295,38.328615,39.326806,19.131474,24.555096,13.199214,15.951463,9.727203,10.714579,7.411801,...,5.328291e+05,4.903806e+07,1.668172e+07,0.0,NaN,1.595851,2.547532e+13,3.065406e+18,211.878167,0.497245
min,-1131.370850,-87.443181,-202.731609,-36.376650,-88.330936,-46.387524,-49.306758,-43.145768,-38.288257,-37.768262,...,0.000000e+00,0.000000e+00,0.000000e+00,0.0,-3.158825e+10,-148.625137,0.000000e+00,0.000000e+00,-3.000000,0.000000
25%,694.982473,47.439627,-30.420775,40.490966,-18.143867,26.212832,-5.415547,16.928649,4.211788,10.952821,...,2.473593e+02,9.677161e+03,5.141539e+03,0.0,-7.150439e+06,-0.076785,3.591583e+04,1.309855e+09,-0.059831,0.000000
50%,816.303475,75.724620,-0.983827,53.835504,0.992457,34.423125,7.687827,22.015233,13.042837,15.171544,...,5.091846e+02,2.154376e+04,1.023633e+04,0.0,9.598652e+05,-0.002857,1.456047e+05,5.729474e+09,0.199766,0.000000
75%,949.003915,101.138536,27.116221,65.404951,21.027308,42.621780,19.318569,27.689323,18.775970,19.048477,...,2.258074e+03,1.068642e+05,4.980395e+04,0.0,1.038031e+07,0.060866,3.430033e+06,1.222085e+11,2.366039,1.000000
max,2065.265392,242.179243,129.581790,132.946777,64.455783,112.663382,58.099707,71.834051,53.528781,64.796099,...,3.963176e+07,3.776215e+09,1.639124e+09,0.0,inf,67.199951,3.692648e+15,4.652774e+20,27450.617153,1.000000


In [14]:
df.to_csv('seismogram_data_66_new.csv', index=False)